In [1]:
from collections import namedtuple
import numpy as np
import pandas as pd

## dealing with dupes


First up, some dummy data where we have repeated emails:

In [80]:
people = [("email@gmail.com", "123-456-7890", "NY"),
          ("email@gmail.com", "321-654-0987", "LA"),
          ("person@gmail.com", "123-789-4567", "WA"),
          ("dummy@gmail.com", "873-345-3456", "MN"),
          ("dummy@gmail.com", "123-345-3456", "NY"), 
          ("email@gmail.com", "000-000-0000", "KY")]

df = pd.DataFrame(data=people, columns=["Email", "Phone", "State"])
df

,Email,Phone,State
0,email@gmail.com,123-456-7890,NY
1,email@gmail.com,321-654-0987,LA
2,person@gmail.com,123-789-4567,WA
3,dummy@gmail.com,873-345-3456,MN
4,dummy@gmail.com,123-345-3456,NY
5,email@gmail.com,000-000-0000,KY


so how to think about dealing with duplicate values in a given col? I'm sure there is a clever way to do this fastly, but heres a (relatively) simple one.

It would be useful to know which one is the first dupe item, so we process that and ignore the others. So first up, I want to mark the first duplicate item.

this looks complicated but what it does is: `gets a list of True vals for all the dupes` and does a AND with `a list of True vals for all first dupes`. 

In [81]:
df["first_dupe"] = df.duplicated("Email", keep=False) & ~df.duplicated("Email", keep="first")
df

,Email,Phone,State,first_dupe
0,email@gmail.com,123-456-7890,NY,True
1,email@gmail.com,321-654-0987,LA,False
2,person@gmail.com,123-789-4567,WA,False
3,dummy@gmail.com,873-345-3456,MN,True
4,dummy@gmail.com,123-345-3456,NY,False
5,email@gmail.com,000-000-0000,KY,False


so now we need a func to do some action with this new found knowledge of the first dupe:

In [82]:
def combine_rows(row, key="Email", cols_to_combine=["Phone", "State"]):
    """takes in a row, looks at the key column
        if its the first dupe, combines the data in cols_to_combine with the other rows with same key
        needs a dataframe with a bool column first_dupe with True if the row is the first dupe"""
    
    if row["first_dupe"] == True:
        # making a df of dupes item
        dupes = df[df[key]==row[key]]
        
        for i, dupe_row in dupes.iloc[1:].iterrows():   # skipping the first row, since thats our first_dupe
            for col in cols_to_combine:
                row[col] += ", " + dupe_row[col]
        # make sure first_dupe doesn't get processed again
        row.first_dupe = False  
    return row
    
df = df.apply(combine_rows, axis=1, result_type=None)
df

,Email,Phone,State,first_dupe
0,email@gmail.com,"123-456-7890, 321-654-0987, 000-000-0000","NY, LA, KY",False
1,email@gmail.com,321-654-0987,LA,False
2,person@gmail.com,123-789-4567,WA,False
3,dummy@gmail.com,"873-345-3456, 123-345-3456","MN, NY",False
4,dummy@gmail.com,123-345-3456,NY,False
5,email@gmail.com,000-000-0000,KY,False


Now we can run the pandas drop_duplicates command:

In [83]:
df.drop_duplicates(subset=["Email"], inplace=True)
df

,Email,Phone,State,first_dupe
0,email@gmail.com,"123-456-7890, 321-654-0987, 000-000-0000","NY, LA, KY",False
2,person@gmail.com,123-789-4567,WA,False
3,dummy@gmail.com,"873-345-3456, 123-345-3456","MN, NY",False


And done! so the whole thing could go into a func called `process_df` which takes in a dateframe and returns the modified one.